In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# ESPN NCAA Men’s Basketball API Endpoint
API_URL = "https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard"


def fetch_schedule(date_obj):
    """Fetch fixtures for a specific date from ESPN."""
    date_str = date_obj.strftime("%Y%m%d")

    params = {
        "dates": date_str,
        "limit": 300,     # Get more events for busy dates
        "groups": 50      # Include NCAA divisions
    }

    for attempt in range(3):  # Retry if ESPN returns nothing
        try:
            resp = requests.get(API_URL, params=params, timeout=10)
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            if attempt < 2:
                print(f"⚠️ Retry {attempt+1}/3 for {date_str}...")
                time.sleep(2)
            else:
                print(f"❌ Failed on {date_str}: {e}")
                return {"events": []}


def parse_events(json_data):
    """Extract structured match data."""
    rows = []

    for event in json_data.get("events", []):
        comp = event.get("competitions", [{}])[0]
        venue = comp.get("venue", {}) or {}
        teams = comp.get("competitors", [])

        home = next((t for t in teams if t.get("homeAway") == "home"), {})
        away = next((t for t in teams if t.get("homeAway") == "away"), {})

        rows.append({
            "Event ID": event.get("id", ""),
            "Date (UTC)": event.get("date", ""),
            "Status": event.get("status", {}).get("type", {}).get("description", ""),
            "Home Team": home.get("team", {}).get("displayName", ""),
            "Home Score": home.get("score", ""),
            "Away Team": away.get("team", {}).get("displayName", ""),
            "Away Score": away.get("score", ""),
            "Venue": venue.get("fullName", ""),
            "City": venue.get("address", {}).get("city", "")
        })

    return rows


if __name__ == "__main__":
    print("\n🏀 NCAA Men's College Basketball Full Season Scraper")
    print("📅 Collecting fixtures...\n")

    start_date = datetime(2025, 11, 1)
    end_date = datetime(2026, 4, 15)

    all_data = []

    current = start_date
    while current <= end_date:
        json_data = fetch_schedule(current)
        rows = parse_events(json_data)

        print(f"{current.date()} → {len(rows)} games scraped")
        all_data.extend(rows)

        time.sleep(0.5)  # prevent API blocking
        current += timedelta(days=1)

    df = pd.DataFrame(all_data)

    # Convert date format
    if "Date (UTC)" in df:
        df["Date"] = pd.to_datetime(df["Date (UTC)"]).dt.date

    # Save output files
    df.to_excel("NCAA_Mens_Full_Season.xlsx", index=False)
    df.to_json("NCAA_Mens_Full_Season.json", indent=4)

    print("\n--------------------------------")
    print(f"✅ TOTAL Games Scraped: {len(df)}")
    print("📁 Saved as:")
    print("   • NCAA_Mens_Full_Season.xlsx")
    print("   • NCAA_Mens_Full_Season.json")
    print("--------------------------------\n")



🏀 NCAA Men's College Basketball Full Season Scraper
📅 Collecting fixtures...

2025-11-01 → 0 games scraped
2025-11-02 → 0 games scraped
2025-11-03 → 169 games scraped
2025-11-04 → 36 games scraped
2025-11-05 → 35 games scraped
2025-11-06 → 44 games scraped
2025-11-07 → 76 games scraped
2025-11-08 → 68 games scraped
2025-11-09 → 34 games scraped
2025-11-10 → 40 games scraped
2025-11-11 → 82 games scraped
2025-11-12 → 54 games scraped
2025-11-13 → 26 games scraped
2025-11-14 → 50 games scraped
2025-11-15 → 82 games scraped
2025-11-16 → 34 games scraped
2025-11-17 → 34 games scraped
2025-11-18 → 68 games scraped
2025-11-19 → 50 games scraped
2025-11-20 → 38 games scraped
2025-11-21 → 73 games scraped
2025-11-22 → 45 games scraped
2025-11-23 → 43 games scraped
2025-11-24 → 62 games scraped
2025-11-25 → 78 games scraped
2025-11-26 → 74 games scraped
2025-11-27 → 18 games scraped
2025-11-28 → 58 games scraped
2025-11-29 → 68 games scraped
2025-11-30 → 33 games scraped
2025-12-01 → 19 games 